#Imports

In [ ]:
import psutil
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0: print('Not connected to a GPU')
else: print(gpu_info)
print()
ram_gb = psutil.virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
if ram_gb < 20: print('Not using a high-RAM runtime')
else: print('You are using a high-RAM runtime!')

Mon Nov  8 22:36:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%matplotlib inline
import random
import math
import time
import re
import os
import shutil
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import collections
import torch
import torchvision as tv
import PIL
import copy
from torch import utils
from torch import optim
from torch.utils import mobile_optimizer

#Parameters

In [ ]:
# DATASET_NAME = 'fairset_good_227'
DATASET_NAME = 'fairset_good_224'
# DATASET_NAME = 'fairset_good_299'
DRIVE_HOME = '/content/drive/MyDrive/sd_cloud_playground/cyberset_riko'

DATASET_DIRECTORY = './{}'.format(DATASET_NAME)
MODELS_DIRECTORY = '{}/models_pt'.format(DRIVE_HOME)
# MODELS_DIRECTORY = './models_pt'

BATCH_SIZE = 4
LEARNING_RATE = 0.001
LEARNING_MOMENTUM = 0.9
DECAY_STEP = 7
DECAY_GAMMA = 0.1
TRAIN_EPOCHS = 25
# TRAIN_EPOCHS = 20
# TRAIN_EPOCHS = 40
# TRAIN_EPOCHS = 1
FAST_LEARNING = False

# NET = 'alexnet'
NET = 'resnet'
# NET = 'inception'
# NET = 'resnet50'
QUANTIZATION = 'aware'
# QUANTIZATION = 'static'
# QUANTIZATION = 'dynamic'
# QUANTIZATION = 'none'
BACKEND_ENGINE = 'qnnpack'
# BACKEND_ENGINE = 'fbgemm'

#Setup

In [ ]:
!cp -u {DRIVE_HOME}/{DATASET_NAME}.zip .
!unzip -nq ./{DATASET_NAME}.zip
!mkdir -p ./models_pt
!pwd
!echo
!ls */

/content

drive/:
MyDrive

fairset_good_224/:
classes.txt  test  train  validation

fairset_good_227/:
classes.txt  test  train  validation

fairset_good_299/:
classes.txt  test  train  validation

models_pt/:
alexnet_fast_none_f32.pt    inception_fast_dynamic_f32.pt
alexnet_full_none_f32.pt    inception_fast_dynamic_i8.ptl
alexnet_full_static_f32.pt  inception_fast_none_f32.pt
checkpoint_f32.pt

sample_data/:
anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


#Utilities

In [ ]:
# show_image(image, label=predicted_class, label_color='red', info=correct_class, info_color='yellow', color=True)
def show_image(image, label='', color=True, info='', label_color='yellow', info_color='yellow'):
    colors={'yellow':((255,255,127)), 'red':(255,63,63), 'green':(63,255,63), 'blue':(63,63,255), 'mono':(224)}
    picture = image.copy()
    plt.figure(figsize=(10,5))
    plt.axis('off')
    if color:
        cv2.putText(picture, label, (0,25), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,0), 6, cv2.LINE_AA)
        cv2.putText(picture, label, (0,25), cv2.FONT_HERSHEY_SIMPLEX, 1,colors[label_color], 2, cv2.LINE_AA)
        cv2.putText(picture, info, (0,55), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,0), 6, cv2.LINE_AA)
        cv2.putText(picture, info, (0,55), cv2.FONT_HERSHEY_SIMPLEX, 1,colors[info_color], 2, cv2.LINE_AA)
        plt.imshow(picture)
    else:
        cv2.putText(picture, label, (0,25), cv2.FONT_HERSHEY_SIMPLEX, 1,(0), 6, cv2.LINE_AA)
        cv2.putText(picture, label, (0,25), cv2.FONT_HERSHEY_SIMPLEX, 1,colors['mono'], 2, cv2.LINE_AA)
        plt.imshow(picture, cmap='gray')
    plt.show()

# dataloaders, classes = get_dataloaders(verbose=True, resize=0)
def get_dataloaders(verbose=True, resize=0):
    if resize > 0:
        data_transform = tv.transforms.Compose([tv.transforms.Resize(resize), tv.transforms.ToTensor(),
                                                tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    else:
        data_transform = tv.transforms.Compose([tv.transforms.ToTensor(), tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    inner_directories = [x for x in os.listdir(DATASET_DIRECTORY) if os.path.isdir('{}/{}'.format(DATASET_DIRECTORY, x))]
    datasets = {x: tv.datasets.ImageFolder(os.path.join(DATASET_DIRECTORY, x), data_transform) for x in inner_directories}
    dataloaders = {x: utils.data.DataLoader(datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=2) for x in inner_directories}
    classes = datasets[inner_directories[0]].classes
    if verbose:
        images_count = {x: len(datasets[x]) for x in inner_directories}
        classes_count = {x: len(datasets[x].classes) for x in inner_directories}
        print('IMAGES:')
        for k, v in images_count.items():
            print(v, k)
        print()
        print('CLASSES:')
        for k, v in classes_count.items():
            print(v, k)
        print()
    return dataloaders, classes

In [ ]:
# predicted_class = query_model(model, image_path, class_names, correct_label=None, resize=0, verbose=True, force_cpu=False)
def query_model(model, image_path, class_names, correct_label=None, resize=0, verbose=True, force_cpu=False):
    model.eval()
    input_image = PIL.Image.open(image_path)
    if resize > 0:
        preprocess = tv.transforms.Compose([tv.transforms.Resize(resize), tv.transforms.ToTensor(),
                                                tv.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    else:
        preprocess = tv.transforms.Compose([tv.transforms.ToTensor(), tv.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)
    if force_cpu:
        input_batch = input_batch.to('cpu')
        model = model.to('cpu')
    else:
        input_batch = input_batch.to('cuda' if torch.cuda.is_available() else 'cpu')
        model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
    with torch.no_grad():
        output = model(input_batch)
    probabilities = torch.nn.functional.softmax(output[0], dim=0)
    top2_prob, top2_id = torch.topk(probabilities, 2)
    predicted_class = classes_names[top2_id[0]]
    second_guess = classes_names[top2_id[1]]
    if verbose:
        if correct_label != None:
            correct_class = correct_label
            if predicted_class == correct_class:
                show_image(np.array(input_image), label='NILF{}'.format(predicted_class), label_color='green')
                print('RIGHT_GUESS -> answer: http://nilf.it/{} | confidence: {:.1f}%\n'.format(predicted_class, top2_prob[0]*100))
            else:
                show_image(np.array(input_image), label='NILF{}'.format(predicted_class), label_color='red',
                           info='NILF{}'.format(correct_class), info_color='yellow')
                print('WRONG_GUESS -> answer: http://nilf.it/{} | confidence: {:.1f}% | second: http://nilf.it/{} | truth: http://nilf.it/{}\n'
                      .format(predicted_class, top2_prob[0]*100, second_guess, correct_class))
        else:
            show_image(np.array(input_image), label='NILF{}'.format(predicted_class))
            print('GUESS -> answer: http://nilf.it/{} | confidence: {:.1f}% | second: http://nilf.it/{}\n'
                  .format(predicted_class, top2_prob[0]*100, second_guess))
    return predicted_class

#Net

In [ ]:
# model, criterion, optimizer, scheduler = build_model(classes_number, net='resnet', quantization='aware', verbose=True)
def build_model(classes_number, net='alexnet', quantization='none', verbose=True):
    if net == 'alexnet':
        model = tv.models.alexnet(pretrained=True)
        if FAST_LEARNING:
            for p in model.parameters():
                p.requires_grad = False
        num_features = model.classifier[6].in_features
        num_classes = classes_number
        model.classifier[6] = torch.nn.Linear(num_features, num_classes)
        model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
        params = [p for (n,p) in model.named_parameters() if p.requires_grad]
        params_names = [n for (n,p) in model.named_parameters() if p.requires_grad]
        if verbose:
            print('PARAMS:')
            for p in params_names:
                print(p)
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = optim.SGD(params, lr=LEARNING_RATE, momentum=LEARNING_MOMENTUM)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=DECAY_STEP, gamma=DECAY_GAMMA)
        if quantization == 'static' or quantization == 'aware':
            model = torch.nn.Sequential(collections.OrderedDict([('quant', torch.quantization.QuantStub()),
                                                                ('alex', model),
                                                                ('dequant', torch.quantization.DeQuantStub())]))
            model.train()
            model.qconfig = torch.quantization.get_default_qat_qconfig(BACKEND_ENGINE)
            torch.backends.quantized.engine = BACKEND_ENGINE
            fusing_modules = []
            fusing_modules.append(['alex.features.0', 'alex.features.1'])
            fusing_modules.append(['alex.features.3', 'alex.features.4'])
            fusing_modules.append(['alex.features.6', 'alex.features.7'])
            fusing_modules.append(['alex.features.8', 'alex.features.9'])
            fusing_modules.append(['alex.features.10', 'alex.features.11'])
            fusing_modules.append(['alex.classifier.1', 'alex.classifier.2'])
            fusing_modules.append(['alex.classifier.4', 'alex.classifier.5'])
            torch.quantization.fuse_modules(model, fusing_modules, inplace=True)
            if quantization == 'aware':
                torch.quantization.prepare_qat(model, inplace=True)
    elif net == 'resnet':
        if quantization == 'static' or quantization == 'aware':
            model = tv.models.quantization.resnet18(pretrained=True)
        else:
            model = tv.models.resnet18(pretrained=True)
        if FAST_LEARNING:
            for p in model.parameters():
                p.requires_grad = False
        num_features = model.fc.in_features
        num_classes = classes_number
        model.fc = torch.nn.Linear(num_features, num_classes)
        model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
        params = [p for (n,p) in model.named_parameters() if p.requires_grad]
        params_names = [n for (n,p) in model.named_parameters() if p.requires_grad]
        if verbose:
            print('PARAMS:')
            for p in params_names:
                print(p)
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = optim.SGD(params, lr=LEARNING_RATE, momentum=LEARNING_MOMENTUM)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=DECAY_STEP, gamma=DECAY_GAMMA)
        if quantization == 'static' or quantization == 'aware':
            model.train()
            if quantization == 'aware':
                model.qconfig = torch.quantization.get_default_qat_qconfig(BACKEND_ENGINE)
                torch.backends.quantized.engine = BACKEND_ENGINE
            fusing_modules = []
            fusing_modules.append(['conv1', 'bn1', 'relu'])
            fusing_modules.append(['layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu'])
            fusing_modules.append(['layer1.0.conv2', 'layer1.0.bn2'])
            fusing_modules.append(['layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu'])
            fusing_modules.append(['layer1.1.conv2', 'layer1.1.bn2'])
            fusing_modules.append(['layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu'])
            fusing_modules.append(['layer2.0.conv2', 'layer2.0.bn2'])
            fusing_modules.append(['layer2.0.downsample.0', 'layer2.0.downsample.1'])
            fusing_modules.append(['layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu'])
            fusing_modules.append(['layer2.1.conv2', 'layer2.1.bn2'])
            fusing_modules.append(['layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.relu'])
            fusing_modules.append(['layer3.0.conv2', 'layer3.0.bn2'])
            fusing_modules.append(['layer3.0.downsample.0', 'layer3.0.downsample.1'])
            fusing_modules.append(['layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.relu'])
            fusing_modules.append(['layer3.1.conv2', 'layer3.1.bn2'])
            fusing_modules.append(['layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.relu'])
            fusing_modules.append(['layer4.0.conv2', 'layer4.0.bn2'])
            fusing_modules.append(['layer4.0.downsample.0', 'layer4.0.downsample.1'])
            fusing_modules.append(['layer4.1.conv1', 'layer4.1.bn1', 'layer4.1.relu'])
            fusing_modules.append(['layer4.1.conv2', 'layer4.1.bn2'])
            torch.quantization.fuse_modules(model, fusing_modules, inplace=True)
            if quantization == 'aware':
                torch.quantization.prepare_qat(model, inplace=True)
    elif net == 'inception':
        model = tv.models.inception_v3(pretrained=True)
        if FAST_LEARNING:
            for p in model.parameters():
                p.requires_grad = False
        num_classes = classes_number
        num_features = model.AuxLogits.fc.in_features
        model.AuxLogits.fc = torch.nn.Linear(num_features, num_classes)
        num_features_2 = model.fc.in_features
        model.fc = torch.nn.Linear(num_features_2, num_classes)
        model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
        params = [p for (n,p) in model.named_parameters() if p.requires_grad]
        params_names = [n for (n,p) in model.named_parameters() if p.requires_grad]
        if verbose:
            print('PARAMS:')
            for p in params_names:
                print(p)
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = optim.SGD(params, lr=LEARNING_RATE, momentum=LEARNING_MOMENTUM)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=DECAY_STEP, gamma=DECAY_GAMMA)
        if quantization == 'static':
            model = torch.nn.Sequential(collections.OrderedDict([('quant', torch.quantization.QuantStub()),
                                                                ('inc', model),
                                                                ('dequant', torch.quantization.DeQuantStub())]))
        elif quantization == 'aware':
            model = None
            criterion = None
            optimizer = None
            scheduler = None
    elif 'resnet50':
        if quantization == 'static' or quantization == 'aware':
            model = tv.models.quantization.resnet50(pretrained=True)
        else:
            model = tv.models.resnet50(pretrained=True)
        if FAST_LEARNING:
            for p in model.parameters():
                p.requires_grad = False
        num_features = model.fc.in_features
        num_classes = classes_number
        model.fc = torch.nn.Linear(num_features, num_classes)
        model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
        params = [p for (n,p) in model.named_parameters() if p.requires_grad]
        params_names = [n for (n,p) in model.named_parameters() if p.requires_grad]
        if verbose:
            print('PARAMS:')
            for p in params_names:
                print(p)
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = optim.SGD(params, lr=LEARNING_RATE, momentum=LEARNING_MOMENTUM)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=DECAY_STEP, gamma=DECAY_GAMMA)
        if quantization == 'aware':
            model.train()
            model.qconfig = torch.quantization.get_default_qat_qconfig(BACKEND_ENGINE)
            torch.backends.quantized.engine = BACKEND_ENGINE
            torch.quantization.prepare_qat(model, inplace=True)
    else:
        model = None
        criterion = None
        optimizer = None
        scheduler = None
    return model, criterion, optimizer, scheduler

In [ ]:
# model, weights = train_model(model, dataloaders, criterion, optimizer, scheduler, inception=True, checkpoint=False)
def train_model(model, dataloaders, criterion, optimizer, scheduler, inception=False, checkpoint=False):
    since = time.time()
    print('TRAINING_STARTED -> elapsed: {}'.format(datetime.timedelta(seconds=round(time.time()-since))))
    best_accuracy = 0.0
    model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
    weights = {'best': copy.deepcopy(model.state_dict()), 'last': copy.deepcopy(model.state_dict())}
    for epoch in range(TRAIN_EPOCHS):
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to('cuda' if torch.cuda.is_available() else 'cpu')
                labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    if inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4 * loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                    _, predictions = torch.max(outputs, 1)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(predictions == labels.data)
            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_accuracy = running_corrects.double() / len(dataloaders[phase].dataset)
            print('{} -> epoch: {}/{} | loss: {:.3} | accuracy: {:.3} | elapsed: {}'
                  .format(phase.upper(), epoch+1, TRAIN_EPOCHS, epoch_loss, epoch_accuracy, datetime.timedelta(seconds=round(time.time()-since))))
            if phase == 'train':
                weights['last'] = copy.deepcopy(model.state_dict())
            if phase == 'validation' and epoch_accuracy > best_accuracy:
                best_accuracy = epoch_accuracy
                weights['best'] = copy.deepcopy(model.state_dict())
                save_weights(model, 'checkpoint')
    print('TRAINING_COMPLETE -> elapsed: {}'.format(datetime.timedelta(seconds=round(time.time()-since))))
    phase = 'test'
    print('TESTING_STARTED -> elapsed: {}'.format(datetime.timedelta(seconds=round(time.time()-since))))
    for state in ['best', 'last']:
        model.load_state_dict(weights[state])
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to('cuda' if torch.cuda.is_available() else 'cpu')
            labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')
            optimizer.zero_grad()
            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, predictions = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(predictions == labels.data)
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_accuracy = running_corrects.double() / len(dataloaders[phase].dataset)
        print('{} -> model: {} | loss: {:.3} | accuracy: {:.3} | elapsed: {}'
              .format(phase.upper(), state, epoch_loss, epoch_accuracy, datetime.timedelta(seconds=round(time.time()-since))))
    model.load_state_dict(weights['best'])
    model.eval()
    print('TESTING_COMPLETE -> elapsed: {}'.format(datetime.timedelta(seconds=round(time.time()-since))))
    return model, weights

In [ ]:
# test_model(model, dataloaders['test'], classes_names, verbose=True, force_cpu=False, times=1)
def test_model(model, dataloader, class_names, verbose=True, force_cpu=False, times=1):
    since = time.time()
    model.eval()
    accuracies = []
    for i in range(times):
        right = 0
        total = 0
        for inputs, labels in dataloader:
            with torch.no_grad():
                if force_cpu:
                    inputs = inputs.to('cpu')
                    labels = labels.to('cpu')
                    model = model.to('cpu')
                else:
                    inputs = inputs.to('cuda' if torch.cuda.is_available() else 'cpu')
                    labels = labels.to('cuda' if torch.cuda.is_available() else 'cpu')
                    model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
                outputs = model(inputs)
                _, predictions = torch.max(outputs, 1)
                predicted_class = class_names[predictions[0]]
                correct_class = class_names[labels[0]]
                if predicted_class == correct_class:
                    right += 1
                elif verbose:
                    print('TEST -> error: http://nilf.it/{} | truth: http://nilf.it/{}'.format(predicted_class, correct_class))
                total += 1
        print('TEST -> correct: {}/{} | accuracy: {:.1f}% | elapsed: {}'.format(right, total, right/total*100, 
                                                                                datetime.timedelta(seconds=round(time.time()-since))))
        accuracies.append(right/total*100)
    if times > 1:
         print('TEST -> average: {:.1f}% | elapsed: {}'.format(np.mean(accuracies), datetime.timedelta(seconds=round(time.time()-since))))

In [ ]:
# save_weights(model, name, torchscript=False, lite=False)
def save_weights(model, name, torchscript=False, lite=False):
    if lite:
        model._save_for_lite_interpreter(os.path.join(MODELS_DIRECTORY, '{}_i8.ptl'.format(name)))
    elif torchscript:
        torch.jit.save(model, os.path.join(MODELS_DIRECTORY, '{}_i8.pt'.format(name)))
    else:
        torch.save(model.state_dict(), os.path.join(MODELS_DIRECTORY, '{}_f32.pt'.format(name)))

# model = load_weights(model, name, torchscript=False, lite=False)
def load_weights(model, name, torchscript=False, lite=False):
    if lite:
        model = torch.jit.load(os.path.join(MODELS_DIRECTORY, '{}_i8.ptl'.format(name)))
    elif torchscript:
        model = torch.jit.load(os.path.join(MODELS_DIRECTORY, '{}_i8.pt'.format(name)))
    else:
        model.load_state_dict(torch.load(os.path.join(MODELS_DIRECTORY, '{}_f32.pt'.format(name))))
    model.eval()
    return model

# script_model = quantize_model(model, quantization='aware')
def quantize_model(model, quantization='aware'):
    model.eval()
    model.to('cpu')
    if quantization == 'aware':
        quantized_model = torch.quantization.convert(model, inplace=False)
    elif quantization == 'static':
        model.qconfig = torch.quantization.get_default_qconfig(BACKEND_ENGINE)
        torch.backends.quantized.engine = BACKEND_ENGINE
        model = torch.quantization.prepare(model, inplace=True)
        quantized_model = torch.quantization.convert(model, inplace=False)
    elif quantization == 'dynamic':
        quantized_model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8, inplace=False)
    script_model = torch.jit.script(quantized_model)
    script_model = mobile_optimizer.optimize_for_mobile(script_model)
    return script_model

#Notes

In [ ]:
# show_image(image, label=predicted_class, label_color='red', info=correct_class, info_color='yellow', color=True)
# dataloaders, classes = get_dataloaders(verbose=True, resize=0)
# predicted_class = query_model(model, image_path, class_names, correct_label=None, resize=0, verbose=True, force_cpu=False)
# model, criterion, optimizer, scheduler = build_model(classes_number, net='resnet', quantization='aware', verbose=True)
# model, weights = train_model(model, dataloaders, criterion, optimizer, scheduler, inception=True, checkpoint=False)
# test_model(model, dataloaders['test'], classes_names, verbose=True, force_cpu=False, times=1)
# save_weights(model, name, torchscript=False, lite=False)
# model = load_weights(model, name, torchscript=False, lite=False)
# script_model = quantize_model(model, quantization='aware')

# test_images = []
# test_labels = []
# for label in os.listdir(os.path.join(DATASET_DIRECTORY, 'test')):
#     for image in os.listdir(os.path.join(DATASET_DIRECTORY, 'test', label)):
#         test_images.append(os.path.join(DATASET_DIRECTORY, 'test', label, image))
#         test_labels.append(label)
# test_set = list(zip(test_images, test_labels))
# random.shuffle(test_set)
# for i, l in test_set:
#     query_model(model, i, l)

#Main

In [ ]:
dataloaders, classes = get_dataloaders()
suffix = 'fast' if FAST_LEARNING else 'full'
pt_lite = True

IMAGES:
975 validation
695 test
39360 train

CLASSES:
656 validation
656 test
656 train



In [ ]:
# model, criterion, optimizer, scheduler = build_model(len(classes), net=NET, quantization=QUANTIZATION, verbose=True)
# model, weights = train_model(model, dataloaders, criterion, optimizer, scheduler, inception=(NET=='inception'), checkpoint=True)

In [ ]:
# save_weights(model, '{}_{}_{}'.format(NET, suffix, QUANTIZATION), torchscript=False, lite=False)
# script_model = quantize_model(model, quantization=QUANTIZATION)
# save_weights(script_model, '{}_{}_{}'.format(NET, suffix, QUANTIZATION), torchscript=True, lite=pt_lite)

In [ ]:
# loaded_model, _, _, _ = build_model(len(classes), net=NET, quantization=QUANTIZATION, verbose=False)
# loaded_model = load_weights(loaded_model, '{}_{}_{}'.format(NET, suffix, QUANTIZATION), torchscript=False, lite=False)
lite_model = load_weights(_, '{}_{}_{}'.format(NET, suffix, QUANTIZATION), torchscript=True, lite=pt_lite)

In [ ]:
# test_model(loaded_model, dataloaders['test'], classes, force_cpu=False, verbose=False, times=10)
test_model(lite_model, dataloaders['test'], classes, force_cpu=True, verbose=True, times=10)

TEST -> error: http://nilf.it/109609 | truth: http://nilf.it/107093
TEST -> error: http://nilf.it/109400 | truth: http://nilf.it/107075
TEST -> error: http://nilf.it/101009 | truth: http://nilf.it/103257
TEST -> error: http://nilf.it/114283 | truth: http://nilf.it/114064
TEST -> error: http://nilf.it/119806 | truth: http://nilf.it/119901
TEST -> error: http://nilf.it/104451 | truth: http://nilf.it/107217
TEST -> correct: 168/174 | accuracy: 96.6% | elapsed: 0:00:39
TEST -> error: http://nilf.it/109400 | truth: http://nilf.it/107075
TEST -> error: http://nilf.it/104451 | truth: http://nilf.it/107217
TEST -> error: http://nilf.it/119806 | truth: http://nilf.it/119901
TEST -> error: http://nilf.it/106148 | truth: http://nilf.it/104454
TEST -> error: http://nilf.it/109609 | truth: http://nilf.it/107093
TEST -> correct: 169/174 | accuracy: 97.1% | elapsed: 0:01:17
TEST -> error: http://nilf.it/111482 | truth: http://nilf.it/113944
TEST -> error: http://nilf.it/106204 | truth: http://nilf.it